<a href="https://colab.research.google.com/github/greyga/TFM-GRG/blob/main/filtroHPOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Carga del archivo de annovar y de genes to phenotype.txt
wannovar_df <- read.csv("/content/exomaBASE+Asthma+decreasedt4+increasedT4 - Copy.csv", header=TRUE)
   head(wannovar_df)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,⋯,Otherinfo.3,Otherinfo.4,Otherinfo.5,Otherinfo.6,Otherinfo.7,Otherinfo.8,Otherinfo.9,Otherinfo.10,Otherinfo.11,Otherinfo.12
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,⋯,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,chr11,116691634,116691634,C,A,exonic,APOA4,NA,nonsynonymous SNV,APOA4:NM_000482:exon3:c.G1140T:p.Q380H,⋯,chr11,116691634,.,C,A,243.48900,.,AB=0.55;ABP=3.44459;AC=1;AF=0.5;AN=2;AO=11;CIGAR=1X;DP=20;DPB=20;DPRA=0;EPP=4.78696;EPPR=5.18177;GTI=0;LEN=1;MEANALT=1;MQM=60;MQMR=60;NS=1;NUMALT=1;ODDS=47.1642;PAIRED=0;PAIREDR=0;PAO=0;PQA=0;PQR=0;PRO=0;QA=392;QR=330;RO=9;RPL=1;RPP=19.0002;RPPR=3.25157;RPR=10;RUN=1;SAF=6;SAP=3.20771;SAR=5;SRF=4;SRP=3.25157;SRR=5;TYPE=snp,GT:DP:AD:RO:QR:AO:QA:GL,"0/1:20:9,11:9:330:11:392:-29.5985,0,-24.0261"
2,chr11,116692334,116692334,C,T,exonic,APOA4,NA,nonsynonymous SNV,APOA4:NM_000482:exon3:c.G440A:p.S147N,⋯,chr11,116692334,.,C,T,543.14100,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=19;CIGAR=1X;DP=19;DPB=19;DPRA=0;EPP=8.61041;EPPR=0;GTI=0;LEN=1;MEANALT=1;MQM=60;MQMR=0;NS=1;NUMALT=1;ODDS=28.0544;PAIRED=0;PAIREDR=0;PAO=0;PQA=0;PQR=0;PRO=0;QA=692;QR=0;RO=0;RPL=13;RPP=8.61041;RPPR=0;RPR=6;RUN=1;SAF=0;SAP=44.2683;SAR=19;SRF=0;SRP=0;SRR=0;TYPE=snp,GT:DP:AD:RO:QR:AO:QA:GL,"1/1:19:0,19:0:0:19:692:-62.6058,-5.71957,0"
3,chr11,116693464,116693464,C,T,exonic,APOA4,NA,synonymous SNV,APOA4:NM_000482:exon2:c.G87A:p.T29T,⋯,chr11,116693464,.,C,T,2610.65000,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=81;CIGAR=1X;DP=81;DPB=81;DPRA=0;EPP=7.5409;EPPR=0;GTI=0;LEN=1;MEANALT=1;MQM=60;MQMR=0;NS=1;NUMALT=1;ODDS=114.005;PAIRED=0;PAIREDR=0;PAO=0;PQA=0;PQR=0;PRO=0;QA=2974;QR=0;RO=0;RPL=45;RPP=5.18177;RPPR=0;RPR=36;RUN=1;SAF=24;SAP=32.2045;SAR=57;SRF=0;SRP=0;SRR=0;TYPE=snp,GT:DP:AD:RO:QR:AO:QA:GL,"1/1:81:0,81:0:0:81:2974:-267.857,-24.3834,0"
4,chr11,116701535,116701535,T,C,exonic,APOC3,NA,synonymous SNV,APOC3:NM_000040:exon3:c.T102C:p.G34G,⋯,chr11,116701535,.,T,C,1246.37000,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=40;CIGAR=1X;DP=40;DPB=40;DPRA=0;EPP=3.22745;EPPR=0;GTI=0;LEN=1;MEANALT=1;MQM=60;MQMR=0;NS=1;NUMALT=1;ODDS=57.1666;PAIRED=0;PAIREDR=0;PAO=0;PQA=0;PQR=0;PRO=0;QA=1419;QR=0;RO=0;RPL=15;RPP=8.43898;RPPR=0;RPR=25;RUN=1;SAF=18;SAP=3.87889;SAR=22;SRF=0;SRP=0;SRR=0;TYPE=snp,GT:DP:AD:RO:QR:AO:QA:GL,"1/1:40:0,40:0:0:40:1419:-127.983,-12.0412,0"
5,chr14,20019984,20019984,G,A,exonic,POTEM,NA,synonymous SNV,POTEM:NM_001145442:exon1:c.C237T:p.N79N,⋯,chr14,20019984,.,G,A,1.37751,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=5;CIGAR=1X;DP=9;DPB=9;DPRA=0;EPP=6.91895;EPPR=3.0103;GTI=0;LEN=1;MEANALT=1;MQM=6;MQMR=5.75;NS=1;NUMALT=1;ODDS=0.985503;PAIRED=0;PAIREDR=0;PAO=0;PQA=0;PQR=0;PRO=0;QA=189;QR=136;RO=4;RPL=4;RPP=6.91895;RPPR=3.0103;RPR=1;RUN=1;SAF=5;SAP=13.8677;SAR=0;SRF=4;SRP=11.6962;SRR=0;TYPE=snp,GT:DP:AD:RO:QR:AO:QA:GL,"1/1:9:4,5:4:136:5:189:-0.633454,-0.583809,0"
6,chr18,56648756,56648756,C,T,exonic,ZNF532,NA,synonymous SNV,"ZNF532:NM_001318726:exon9:c.C3318T:p.H1106H,ZNF532:NM_001318727:exon9:c.C3318T:p.H1106H,ZNF532:NM_001318728:exon10:c.C3318T:p.H1106H,ZNF532:NM_018181:exon10:c.C3318T:p.H1106H",⋯,chr18,56648756,.,C,T,3.49454,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;DPB=2;DPRA=0;EPP=7.35324;EPPR=0;GTI=0;LEN=1;MEANALT=1;MQM=10;MQMR=0;NS=1;NUMALT=1;ODDS=0.211183;PAIRED=0;PAIREDR=0;PAO=0;PQA=0;PQR=0;PRO=0;QA=71;QR=0;RO=0;RPL=0;RPP=7.35324;RPPR=0;RPR=2;RUN=1;SAF=2;SAP=7.35324;SAR=0;SRF=0;SRP=0;SRR=0;TYPE=snp,GT:DP:AD:RO:QR:AO:QA:GL,"1/1:2:0,2:0:0:2:71:-1.8979,-0.60206,0"


In [ ]:
library(dplyr)
# Cambiar el nombre de la columna "AAChange.refGene" a "Transcript_exon_HGVSc._HGVSp."
colnames(wannovar_df)[which(colnames(wannovar_df) == "AAChange.refGene")] <- "Transcript_exon_HGVSc._HGVSp."

# Cambiar el nombre de la columna "Otherinfo" a "Zygosity" y cambiar los valores "het" a "heterozygous" y "hom" a "homozygous"
wannovar_df <- wannovar_df %>%
  mutate(Zygosity = ifelse(Otherinfo == "het", "heterozygous", ifelse(Otherinfo == "hom", "homozygous", NA))) %>%
  select(Gene.refGene, Zygosity, Transcript_exon_HGVSc._HGVSp., everything(), -Otherinfo)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
#seleccionar columnas deseadas
subset_df <- subset(wannovar_df, select = c("Gene.refGene", "Zygosity", "Transcript_exon_HGVSc._HGVSp.", "dbSNP", "Chr", "Start", "End", "Ref", "Alt", "Func.refGene", "ExonicFunc.refGene", "X1000G_ALL", "X1000G_AFR", "X1000G_AMR", "X1000G_EAS", "X1000G_EUR", "X1000G_SAS", "ExAC_Freq", "ExAC_AFR", "ExAC_AMR", "ExAC_EAS", "ExAC_FIN", "ExAC_NFE", "ExAC_OTH", "ExAC_SAS", "ClinVar_SIG", "ClinVar_DIS", "GWAS_OR", "GWAS_BETA", "GWAS_PUBMED", "GWAS_SNP", "GWAS_P", "SIFT_score", "SIFT_converted_rankscore", "SIFT_pred", "Polyphen2_HDIV_score", "Polyphen2_HDIV_rankscore", "Polyphen2_HDIV_pred", "Polyphen2_HVAR_score", "Polyphen2_HVAR_rankscore", "Polyphen2_HVAR_pred", "LRT_score", "LRT_converted_rankscore", "LRT_pred", "MutationTaster_score", "MutationTaster_converted_rankscore", "MutationTaster_pred", "MutationAssessor_score", "MutationAssessor_score_rankscore", "MutationAssessor_pred", "FATHMM_score", "FATHMM_converted_rankscore", "FATHMM_pred", "CADD_raw_rankscore", "CADD_phred", "DANN_score", "DANN_rankscore", "GERP.._RS", "gnomAD_exome_ALL", "gnomAD_exome_AFR", "gnomAD_exome_AMR", "gnomAD_exome_ASJ", "gnomAD_exome_EAS", "gnomAD_exome_FIN", "gnomAD_exome_NFE", "gnomAD_exome_OTH", "gnomAD_exome_SAS", "gnomAD_genome_ALL", "gnomAD_genome_AFR", "gnomAD_genome_AMR", "gnomAD_genome_ASJ", "gnomAD_genome_EAS", "gnomAD_genome_FIN", "gnomAD_genome_NFE", "gnomAD_genome_OTH"))
subset_df

,Gene.refGene,Zygosity,Transcript_exon_HGVSc._HGVSp.,dbSNP,Chr,Start,End,Ref,Alt,Func.refGene,⋯,gnomAD_exome_OTH,gnomAD_exome_SAS,gnomAD_genome_ALL,gnomAD_genome_AFR,gnomAD_genome_AMR,gnomAD_genome_ASJ,gnomAD_genome_EAS,gnomAD_genome_FIN,gnomAD_genome_NFE,gnomAD_genome_OTH
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,APOA4,heterozygous,APOA4:NM_000482:exon3:c.G1140T:p.Q380H,rs5110,chr11,116691634,116691634,C,A,exonic,⋯,0.0614,0.0188,0.0481,0.0159,0.0525,0.0497,0.0006,0.0478,0.0721,0.0429
2,APOA4,homozygous,APOA4:NM_000482:exon3:c.G440A:p.S147N,rs5104,chr11,116692334,116692334,C,T,exonic,⋯,0.8176,0.675,0.8375,0.8835,0.7811,0.7483,0.6796,0.759,0.8528,0.8108
3,APOA4,homozygous,APOA4:NM_000482:exon2:c.G87A:p.T29T,rs5092,chr11,116693464,116693464,C,T,exonic,⋯,0.7862,0.6327,0.8029,0.8525,0.7685,0.7384,0.6343,0.7282,0.8145,0.7781
4,APOC3,homozygous,APOC3:NM_000040:exon3:c.T102C:p.G34G,rs4520,chr11,116701535,116701535,T,C,exonic,⋯,0.6639,0.5632,0.6899,0.7303,0.5601,0.6589,0.4356,0.6282,0.7191,0.6451
5,POTEM,homozygous,POTEM:NM_001145442:exon1:c.C237T:p.N79N,rs200569982,chr14,20019984,20019984,G,A,exonic,⋯,0,0.0002,0.0099,0.0249,0.0013,0.0035,0.004,0.0041,0.0051,0.0068
6,ZNF532,homozygous,"ZNF532:NM_001318726:exon9:c.C3318T:p.H1106H,ZNF532:NM_001318727:exon9:c.C3318T:p.H1106H,ZNF532:NM_001318728:exon10:c.C3318T:p.H1106H,ZNF532:NM_018181:exon10:c.C3318T:p.H1106H",rs61733373,chr18,56648756,56648756,C,T,exonic,⋯,0.131,0.1466,0.1135,0.0228,0.0585,0.2133,0.0191,0.2135,0.151,0.1656
7,ZNF532,homozygous,"ZNF532:NM_001318726:exon9:c.C3347T:p.P1116L,ZNF532:NM_001318727:exon9:c.C3347T:p.P1116L,ZNF532:NM_001318728:exon10:c.C3347T:p.P1116L,ZNF532:NM_018181:exon10:c.C3347T:p.P1116L",rs760239915,chr18,56648785,56648785,C,T,exonic,⋯,0,0,.,.,.,.,.,.,.,.
8,ZNF532,homozygous,"ZNF532:NM_001318726:exon9:c.A3401T:p.E1134V,ZNF532:NM_001318727:exon9:c.A3401T:p.E1134V,ZNF532:NM_001318728:exon10:c.A3401T:p.E1134V,ZNF532:NM_018181:exon10:c.A3401T:p.E1134V",.,chr18,56648839,56648839,A,T,exonic,⋯,.,.,.,.,.,.,.,.,.,.
9,ACTR2,homozygous,"ACTR2:NM_005722:exon7:c.C791T:p.A264V,ACTR2:NM_001005386:exon8:c.C806T:p.A269V",.,chr2,65488436,65488436,C,T,exonic,⋯,0,0,.,.,.,.,.,.,.,.


In [ ]:
# Leer el archivo genes_to_phenotype.txt
genes_pheno <- read.delim("/content/genes_to_phenotype.txt", header=FALSE, comment.char="#")

# Seleccionar solo las columnas necesarias
genes_pheno_sub <- genes_pheno[, c("V2", "V3", "V4", "V9")]
genes_pheno_sub
# Renombrar las columnas
colnames(genes_pheno_sub) <- c("gene_id", "hpo_id", "hpo_desc", "omim_orpha")

# Combinar la información de hpo_id y hpo_desc en una sola columna separados por una coma
genes_pheno_sub$hpo <- paste(genes_pheno_sub$hpo_id, genes_pheno_sub$hpo_desc, sep = ", ")

# Eliminar las columnas hpo_id y hpo_name originales
genes_pheno_sub <- genes_pheno_sub[, !names(genes_pheno_sub) %in% c("hpo_id", "hpo_desc")]

# Agrupar por gene_id y concatenar los HPO y los códigos OMIM_OPRHA separados por ;
genes_pheno_aggr <- aggregate(cbind(hpo, omim_orpha) ~ gene_id, data = genes_pheno_sub, FUN = function(x) paste(x, collapse = "; "))

# Cambiar el nombre de la columna gene_symbol a Gene.refGene en la tabla de HPO
 colnames(genes_pheno_aggr)[1] <- "Gene.refGene"

V2,V3,V4,V9
<chr>,<chr>,<chr>,<chr>
CLPP,HP:0000013,Hypoplasia of the uterus,OMIM:614129
CLPP,HP:0000815,Hypergonadotropic hypogonadism,OMIM:614129
CLPP,HP:0000786,Primary amenorrhea,OMIM:614129
CLPP,HP:0000007,Autosomal recessive inheritance,OMIM:614129
CLPP,HP:0000252,Microcephaly,OMIM:614129
CLPP,HP:0001250,Seizure,OMIM:614129
CLPP,HP:0004322,Short stature,OMIM:614129
CLPP,HP:0008527,Congenital sensorineural hearing impairment,OMIM:614129
MKKS,HP:0000218,High palate,ORPHA:2473


In [ ]:
## Realizar el merge
merged_df <- merge(subset_df, genes_pheno_aggr, by = "Gene.refGene", all.x = TRUE)
# Reemplazar los valores NA por "NO HPO" en merged_df
 if (anyNA(merged_df$hpo)) {
   merged_df$hpo[is.na(merged_df$hpo)] <- "NO HPO"}

# Reemplazar los valores NA por "NO OMIM_OPRHA" en merged_df
 if (anyNA(merged_df$omim_orpha)) {
   merged_df$omim_orpha[is.na(merged_df$omim_orpha)] <- "NO OMIM/OPRHA"}
merged_df

Gene.refGene Zygosity    
1   A4GALT       heterozygous
2   A4GALT       heterozygous
3   A4GALT       heterozygous
4   ACO2         heterozygous
5   ACO2         heterozygous
6   ACTR2        homozygous  
7   ADA2         homozygous  
8   ADRB2        homozygous  
9   AIFM3        homozygous  
10  ALG12        heterozygous
11  ALG12        heterozygous
12  AP1B1        heterozygous
13  AP1B1        homozygous  
14  AP1B1        heterozygous
15  APOA4        heterozygous
16  APOA4        homozygous  
17  APOA4        homozygous  
18  APOBEC3B     heterozygous
19  APOBEC3B     homozygous  
20  APOBEC3B     homozygous  
21  APOBEC3B     homozygous  
22  APOBEC3F     heterozygous
23  APOBEC3F     heterozygous
24  APOBEC3F     homozygous  
25  APOBEC3F     heterozygous
26  APOBEC3H     heterozygous
27  APOBEC3H     homozygous  
28  APOBEC3H     homozygous  
29  APOBEC3H     homozygous  
30  APOBEC3H     homozygous  
⋮   ⋮            ⋮           
452 UPK3A        homozygous  
453 USP18        heterozygous
454 WBP2NL       heterozygous
455 WBP2NL       homozygous  
456 YDJC         heterozygous
457 ZBED4        heterozygous
458 ZBED4        heterozygous
459 ZBED4        heterozygous
460 ZBED4        heterozygous
461 ZBED4        heterozygous
462 ZBED4        heterozygous
463 ZC3H7B       heterozygous
464 ZC3H7B       heterozygous
465 ZNF280A      heterozygous
466 ZNF280A      homozygous  
467 ZNF280A      homozygous  
468 ZNF280A      homozygous  
469 ZNF280A      homozygous  
470 ZNF280A      homozygous  
471 ZNF280A      homozygous  
472 ZNF280A      homozygous  
473 ZNF280A      homozygous  
474 ZNF280B      homozygous  
475 ZNF280B      homozygous  
476 ZNF280B      homozygous  
477 ZNF532       homozygous  
478 ZNF532       homozygous  
479 ZNF532       homozygous  
480 ZNF74        heterozygous
481 ZNRF3        homozygous  
    Transcript_exon_HGVSc._HGVSp.                                                                                                                                                                   
1   A4GALT:NM_001318038:exon3:c.A109G:p.M37V,A4GALT:NM_017436:exon3:c.A109G:p.M37V                                                                                                                  
2   A4GALT:NM_001318038:exon3:c.G987A:p.T329T,A4GALT:NM_017436:exon3:c.G987A:p.T329T                                                                                                                
3   A4GALT:NM_001318038:exon3:c.C903G:p.P301P,A4GALT:NM_017436:exon3:c.C903G:p.P301P                                                                                                                
4   ACO2:NM_001098:exon5:c.C670T:p.L224L                                                                                                                                                            
5   ACO2:NM_001098:exon3:c.A192C:p.T64T                                                                                                                                                             
6   ACTR2:NM_005722:exon7:c.C791T:p.A264V,ACTR2:NM_001005386:exon8:c.C806T:p.A269V                                                                                                                  
7   ADA2:NM_001282225:exon2:c.C159T:p.N53N,ADA2:NM_001282226:exon2:c.C159T:p.N53N,ADA2:NM_001282227:exon2:c.C33T:p.N11N,ADA2:NM_001282228:exon2:c.C33T:p.N11N                                       
8   ADRB2:NM_000024:exon1:c.G420C:p.K140N                                                                                                                                                           
9   AIFM3:NM_001018060:exon12:c.A1134T:p.R378R,AIFM3:NM_001146288:exon12:c.A1152T:p.R384R,AIFM3:NM_144704:exon12:c.A1134T:p.R378R                                                                   
10  ALG12:NM_024105:exon9:c.A1177G:p.I393V                                                                                                                                    

In [ ]:
#ver tipo de las variables
str(merged_df)

'data.frame':	481 obs. of  77 variables:
 $ Gene.refGene                      : chr  "A4GALT" "A4GALT" "A4GALT" "ACO2" ...
 $ Zygosity                          : chr  "heterozygous" "heterozygous" "heterozygous" "heterozygous" ...
 $ Transcript_exon_HGVSc._HGVSp.     : chr  "A4GALT:NM_001318038:exon3:c.A109G:p.M37V,A4GALT:NM_017436:exon3:c.A109G:p.M37V" "A4GALT:NM_001318038:exon3:c.G987A:p.T329T,A4GALT:NM_017436:exon3:c.G987A:p.T329T" "A4GALT:NM_001318038:exon3:c.C903G:p.P301P,A4GALT:NM_017436:exon3:c.C903G:p.P301P" "ACO2:NM_001098:exon5:c.C670T:p.L224L" ...
 $ dbSNP                             : chr  "rs11541159" "rs9623659" "rs6002904" "rs1799932" ...
 $ Chr                               : chr  "chr22" "chr22" "chr22" "chr22" ...
 $ Start                             : int  43089849 43088971 43089055 41911525 41903813 65488436 17690409 148206814 21331043 50297888 ...
 $ End                               : int  43089849 43088971 43089055 41911525 41903813 65488436 17690409 148206814 21

In [ ]:
#cambiar las columnas de las predicciones a caategorias

merged_df$SIFT_pred <- factor(merged_df$SIFT_pred, levels = c(".", "T", "D"), labels = c("Tolerated", "Damaging", "Deleterious"))
merged_df$Polyphen2_HDIV_pred <- factor(merged_df$Polyphen2_HDIV_pred, levels = c(".", "B", "D", "P"), labels = c("Benign", "Possibly damaging", "Probably damaging", "Damaging"))
merged_df$Polyphen2_HVAR_pred <- factor(merged_df$Polyphen2_HVAR_pred, levels = c(".", "B", "D", "P"), labels = c("Benign", "Possibly damaging", "Probably damaging", "Damaging"))
merged_df$MutationTaster_pred <- factor(merged_df$MutationTaster_pred, levels = c(".", "A", "D", "N", "P"), labels = c("Polymorphism", "Disease causing automatic", "Disease causing", "Polymorphism automatic", "Polymorphism"))
merged_df$MutationAssessor_pred <- factor(merged_df$MutationAssessor_pred, levels = c(".", "H", "M", "L", "N"), labels = c("Neutral", "High", "Medium", "Low", "Not provided"))
merged_df$FATHMM_pred <- factor(merged_df$FATHMM_pred, levels = c(".", "T", "D"), labels = c("Tolerated", "Damaging", "Deleterious"))
merged_df$ClinVar_SIG <- factor(merged_df$ClinVar_SIG, levels = c("Uncertain significance", "Benign", "Likely benign", "Likely pathogenic", "Pathogenic", "Drug response", "Histocompatibility", "Conflicting interpretations of pathogenicity"))

In [ ]:
# cambiar frecuencias y scores as numeric
merged_df$X1000G_ALL <- as.numeric(merged_df$X1000G_ALL)
merged_df$X1000G_AFR <- as.numeric(merged_df$X1000G_AFR)
merged_df$X1000G_AMR <- as.numeric(merged_df$X1000G_AMR)
merged_df$X1000G_EAS <- as.numeric(merged_df$X1000G_EAS)
merged_df$X1000G_EUR <- as.numeric(merged_df$X1000G_EUR)
merged_df$X1000G_SAS <- as.numeric(merged_df$X1000G_SAS)
merged_df$ExAC_Freq <- as.numeric(merged_df$ExAC_Freq)
merged_df$ExAC_AFR <- as.numeric(merged_df$ExAC_AFR)
merged_df$ExAC_AMR <- as.numeric(merged_df$ExAC_AMR)
merged_df$ExAC_EAS <- as.numeric(merged_df$ExAC_EAS)
merged_df$ExAC_FIN <- as.numeric(merged_df$ExAC_FIN)
merged_df$ExAC_NFE <- as.numeric(merged_df$ExAC_NFE)
merged_df$ExAC_OTH <- as.numeric(merged_df$ExAC_OTH)
merged_df$ExAC_SAS <- as.numeric(merged_df$ExAC_SAS)
merged_df$GWAS_OR <- as.numeric(merged_df$GWAS_OR)
merged_df$GWAS_BETA <- as.numeric(merged_df$GWAS_BETA)
merged_df$GWAS_P <- as.numeric(merged_df$GWAS_P)
merged_df$SIFT_score <- as.numeric(merged_df$SIFT_score)
merged_df$SIFT_converted_rankscore <- as.numeric(merged_df$SIFT_converted_rankscore)
merged_df$Polyphen2_HDIV_score <- as.numeric(merged_df$Polyphen2_HDIV_score)
merged_df$Polyphen2_HDIV_rankscore <- as.numeric(merged_df$Polyphen2_HDIV_rankscore)
merged_df$Polyphen2_HVAR_score <- as.numeric(merged_df$Polyphen2_HVAR_score)
merged_df$Polyphen2_HVAR_rankscore <- as.numeric(merged_df$Polyphen2_HVAR_rankscore)
merged_df$LRT_score <- as.numeric(merged_df$LRT_score)
merged_df$LRT_converted_rankscore <- as.numeric(merged_df$LRT_converted_rankscore)
merged_df$MutationTaster_score <- as.numeric(merged_df$MutationTaster_score)
merged_df$MutationTaster_converted_rankscore <- as.numeric(merged_df$MutationTaster_converted_rankscore)
merged_df$MutationAssessor_score <- as.numeric(merged_df$MutationAssessor_score)
merged_df$MutationAssessor_score_rankscore <- as.numeric(merged_df$MutationAssessor_score_rankscore)
merged_df$FATHMM_score <- as.numeric(merged_df$FATHMM_score)
merged_df$FATHMM_converted_rankscore <- as.numeric(merged_df$FATHMM_converted_rankscore)
merged_df$CADD_raw_rankscore <- as.numeric(merged_df$CADD_raw_rankscore)
merged_df$CADD_phred <- as.numeric(merged_df$CADD_phred)
merged_df$GERP.._RS <- as.numeric(as.character(merged_df$GERP.._RS))
merged_df$gnomAD_exome_ALL <- as.numeric(as.character(merged_df$gnomAD_exome_ALL))
merged_df$gnomAD_exome_AFR <- as.numeric(as.character(merged_df$gnomAD_exome_AFR))
merged_df$gnomAD_exome_AMR <- as.numeric(as.character(merged_df$gnomAD_exome_AMR))
merged_df$gnomAD_exome_ASJ <- as.numeric(as.character(merged_df$gnomAD_exome_ASJ))
merged_df$gnomAD_exome_EAS <- as.numeric(as.character(merged_df$gnomAD_exome_EAS))
merged_df$gnomAD_exome_FIN <- as.numeric(as.character(merged_df$gnomAD_exome_FIN))
merged_df$gnomAD_exome_NFE <- as.numeric(as.character(merged_df$gnomAD_exome_NFE))
merged_df$gnomAD_exome_OTH <- as.numeric(as.character(merged_df$gnomAD_exome_OTH))
merged_df$gnomAD_exome_SAS <- as.numeric(as.character(merged_df$gnomAD_exome_SAS))
merged_df$gnomAD_genome_ALL <- as.numeric(as.character(merged_df$gnomAD_genome_ALL))
merged_df$gnomAD_genome_AFR <- as.numeric(as.character(merged_df$gnomAD_genome_AFR))
merged_df$gnomAD_genome_AMR <- as.numeric(as.character(merged_df$gnomAD_genome_AMR))
merged_df$gnomAD_genome_ASJ <- as.numeric(as.character(merged_df$gnomAD_genome_ASJ))
merged_df$gnomAD_genome_EAS <- as.numeric(as.character(merged_df$gnomAD_genome_EAS))
merged_df$gnomAD_genome_FIN <- as.numeric(as.character(merged_df$gnomAD_genome_FIN))
merged_df$gnomAD_genome_NFE <- as.numeric(as.character(merged_df$gnomAD_genome_NFE))
merged_df$gnomAD_genome_OTH <- as.numeric(as.character(merged_df$gnomAD_genome_OTH))

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(e

In [ ]:
colnames(merged_df)
names(merged_df)


[1] "Gene.refGene"                       "Zygosity"                          
 [3] "Transcript_exon_HGVSc._HGVSp."      "dbSNP"                             
 [5] "Chr"                                "Start"                             
 [7] "End"                                "Ref"                               
 [9] "Alt"                                "Func.refGene"                      
[11] "ExonicFunc.refGene"                 "X1000G_ALL"                        
[13] "X1000G_AFR"                         "X1000G_AMR"                        
[15] "X1000G_EAS"                         "X1000G_EUR"                        
[17] "X1000G_SAS"                         "ExAC_Freq"                         
[19] "ExAC_AFR"                           "ExAC_AMR"                          
[21] "ExAC_EAS"                           "ExAC_FIN"                          
[23] "ExAC_NFE"                           "ExAC_OTH"                          
[25] "ExAC_SAS"                           "ClinVar_SIG"                       
[27] "ClinVar_DIS"                        "GWAS_OR"                           
[29] "GWAS_BETA"                          "GWAS_PUBMED"                       
[31] "GWAS_SNP"                           "GWAS_P"                            
[33] "SIFT_score"                         "SIFT_converted_rankscore"          
[35] "SIFT_pred"                          "Polyphen2_HDIV_score"              
[37] "Polyphen2_HDIV_rankscore"           "Polyphen2_HDIV_pred"               
[39] "Polyphen2_HVAR_score"               "Polyphen2_HVAR_rankscore"          
[41] "Polyphen2_HVAR_pred"                "LRT_score"                         
[43] "LRT_converted_rankscore"            "LRT_pred"                          
[45] "MutationTaster_score"               "MutationTaster_converted_rankscore"
[47] "MutationTaster_pred"                "MutationAssessor_score"            
[49] "MutationAssessor_score_rankscore"   "MutationAssessor_pred"             
[51] "FATHMM_score"                       "FATHMM_converted_rankscore"        
[53] "FATHMM_pred"                        "CADD_raw_rankscore"                
[55] "CADD_phred"                         "DANN_score"                        
[57] "DANN_rankscore"                     "GERP.._RS"                         
[59] "gnomAD_exome_ALL"                   "gnomAD_exome_AFR"                  
[61] "gnomAD_exome_AMR"                   "gnomAD_exome_ASJ"                  
[63] "gnomAD_exome_EAS"                   "gnomAD_exome_FIN"                  
[65] "gnomAD_exome_NFE"                   "gnomAD_exome_OTH"                  
[67] "gnomAD_exome_SAS"                   "gnomAD_genome_ALL"                 
[69] "gnomAD_genome_AFR"                  "gnomAD_genome_AMR"                 
[71] "gnomAD_genome_ASJ"                  "gnomAD_genome_EAS"                 
[73] "gnomAD_genome_FIN"                  "gnomAD_genome_NFE"                 
[75] "gnomAD_genome_OTH"                  "hpo"                               
[77] "omim_orpha"

[1] "Gene.refGene"                       "Zygosity"                          
 [3] "Transcript_exon_HGVSc._HGVSp."      "dbSNP"                             
 [5] "Chr"                                "Start"                             
 [7] "End"                                "Ref"                               
 [9] "Alt"                                "Func.refGene"                      
[11] "ExonicFunc.refGene"                 "X1000G_ALL"                        
[13] "X1000G_AFR"                         "X1000G_AMR"                        
[15] "X1000G_EAS"                         "X1000G_EUR"                        
[17] "X1000G_SAS"                         "ExAC_Freq"                         
[19] "ExAC_AFR"                           "ExAC_AMR"                          
[21] "ExAC_EAS"                           "ExAC_FIN"                          
[23] "ExAC_NFE"                           "ExAC_OTH"                          
[25] "ExAC_SAS"                           "ClinVar_SIG"                       
[27] "ClinVar_DIS"                        "GWAS_OR"                           
[29] "GWAS_BETA"                          "GWAS_PUBMED"                       
[31] "GWAS_SNP"                           "GWAS_P"                            
[33] "SIFT_score"                         "SIFT_converted_rankscore"          
[35] "SIFT_pred"                          "Polyphen2_HDIV_score"              
[37] "Polyphen2_HDIV_rankscore"           "Polyphen2_HDIV_pred"               
[39] "Polyphen2_HVAR_score"               "Polyphen2_HVAR_rankscore"          
[41] "Polyphen2_HVAR_pred"                "LRT_score"                         
[43] "LRT_converted_rankscore"            "LRT_pred"                          
[45] "MutationTaster_score"               "MutationTaster_converted_rankscore"
[47] "MutationTaster_pred"                "MutationAssessor_score"            
[49] "MutationAssessor_score_rankscore"   "MutationAssessor_pred"             
[51] "FATHMM_score"                       "FATHMM_converted_rankscore"        
[53] "FATHMM_pred"                        "CADD_raw_rankscore"                
[55] "CADD_phred"                         "DANN_score"                        
[57] "DANN_rankscore"                     "GERP.._RS"                         
[59] "gnomAD_exome_ALL"                   "gnomAD_exome_AFR"                  
[61] "gnomAD_exome_AMR"                   "gnomAD_exome_ASJ"                  
[63] "gnomAD_exome_EAS"                   "gnomAD_exome_FIN"                  
[65] "gnomAD_exome_NFE"                   "gnomAD_exome_OTH"                  
[67] "gnomAD_exome_SAS"                   "gnomAD_genome_ALL"                 
[69] "gnomAD_genome_AFR"                  "gnomAD_genome_AMR"                 
[71] "gnomAD_genome_ASJ"                  "gnomAD_genome_EAS"                 
[73] "gnomAD_genome_FIN"                  "gnomAD_genome_NFE"                 
[75] "gnomAD_genome_OTH"                  "hpo"                               
[77] "omim_orpha"

In [ ]:
install.packages("stringr")
library(stringr)

# crear una función que extraiga los códigos HPO de una cadena de texto
extract_hpo <- function(x) {
  hpo_codes <- str_extract_all(x, "(HP:\\d+)")
  if (length(hpo_codes) > 0) {
    paste(hpo_codes[[1]], collapse = ", ")
  } else {
    ""
  }
}

# aplicar la función a la columna 'hpo' para crear una nueva columna 'HPO codes gene'
merged_df$`HPO codes gene` <- sapply(merged_df$hpo, extract_hpo)
head(merged_df)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



,Gene.refGene,Zygosity,Transcript_exon_HGVSc._HGVSp.,dbSNP,Chr,Start,End,Ref,Alt,Func.refGene,⋯,gnomAD_genome_AFR,gnomAD_genome_AMR,gnomAD_genome_ASJ,gnomAD_genome_EAS,gnomAD_genome_FIN,gnomAD_genome_NFE,gnomAD_genome_OTH,hpo,omim_orpha,HPO codes gene
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,A4GALT,heterozygous,"A4GALT:NM_001318038:exon3:c.A109G:p.M37V,A4GALT:NM_017436:exon3:c.A109G:p.M37V",rs11541159,chr22,43089849,43089849,T,C,exonic,⋯,0.4869,0.3732,0.4000,0.1434,0.2883,0.3477,0.3286,"HP:0000006, Autosomal dominant inheritance; HP:0010970, Blood group antigen abnormality",OMIM:111400; OMIM:111400,"HP:0000006, HP:0010970"
2,A4GALT,heterozygous,"A4GALT:NM_001318038:exon3:c.G987A:p.T329T,A4GALT:NM_017436:exon3:c.G987A:p.T329T",rs9623659,chr22,43088971,43088971,C,T,exonic,⋯,0.4834,0.3786,0.4040,0.1422,0.2886,0.3480,0.3272,"HP:0000006, Autosomal dominant inheritance; HP:0010970, Blood group antigen abnormality",OMIM:111400; OMIM:111400,"HP:0000006, HP:0010970"
3,A4GALT,heterozygous,"A4GALT:NM_001318038:exon3:c.C903G:p.P301P,A4GALT:NM_017436:exon3:c.C903G:p.P301P",rs6002904,chr22,43089055,43089055,G,C,exonic,⋯,0.8364,0.6172,0.5767,0.7171,0.5956,0.5703,0.5932,"HP:0000006, Autosomal dominant inheritance; HP:0010970, Blood group antigen abnormality",OMIM:111400; OMIM:111400,"HP:0000006, HP:0010970"
4,ACO2,heterozygous,ACO2:NM_001098:exon5:c.C670T:p.L224L,rs1799932,chr22,41911525,41911525,C,T,exonic,⋯,0.1564,0.2506,0.4233,0.1634,0.4417,0.5030,0.4436,"HP:0000007, Autosomal recessive inheritance; HP:0003621, Juvenile onset; HP:0000543, Optic disc pallor; HP:0007663, Reduced visual acuity; HP:0030528, Paracentral scotoma; HP:0000648, Optic atrophy; HP:0000642, Red-green dyschromatopsia; HP:0011463, Childhood onset; HP:0001252, Hypotonia; HP:0002079, Hypoplasia of the corpus callosum; HP:0003593, Infantile onset; HP:0000486, Strabismus; HP:0002305, Athetosis; HP:0000253, Progressive microcephaly; HP:0007359, Focal-onset seizure; HP:0001284, Areflexia; HP:0011344, Severe global developmental delay; HP:0000556, Retinal dystrophy; HP:0000007, Autosomal recessive inheritance; HP:0000407, Sensorineural hearing impairment; HP:0000648, Optic atrophy; HP:0001251, Ataxia; HP:0010864, Intellectual disability, severe; HP:0001272, Cerebellar atrophy; HP:0007108, Demyelinating peripheral neuropathy; HP:0002120, Cerebral cortical atrophy; HP:0001265, Hyporeflexia; HP:0002069, Bilateral tonic-clonic seizure; HP:0001508, Failure to thrive; HP:0000639, Nystagmus",OMIM:616289; OMIM:616289; OMIM:616289; OMIM:616289; OMIM:616289; OMIM:616289; OMIM:616289; OMIM:616289; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559,"HP:0000007, HP:0003621, HP:0000543, HP:0007663, HP:0030528, HP:0000648, HP:0000642, HP:0011463, HP:0001252, HP:0002079, HP:0003593, HP:0000486, HP:0002305, HP:0000253, HP:0007359, HP:0001284, HP:0011344, HP:0000556, HP:0000007, HP:0000407, HP:0000648, HP:0001251, HP:0010864, HP:0001272, HP:0007108, HP:0002120, HP:0001265, HP:0002069, HP:0001508, HP:0000639"
5,ACO2,heterozygous,ACO2:NM_001098:exon3:c.A192C:p.T64T,rs137831,chr22,41903813,41903813,A,C,exonic,⋯,0.3553,0.5800,0.2947,0.5795,0.2552,0.1987,0.2520,"HP:0000007, Autosomal recessive inheritance; HP:0003621, Juvenile onset; HP:0000543, Optic disc pallor; HP:0007663, Reduced visual acuity; HP:0030528, Paracentral scotoma; HP:0000648, Optic atrophy; HP:0000642, Red-green dyschromatopsia; HP:0011463, Childhood onset; HP:0001252, Hypotonia; HP:0002079, Hypoplasia of the corpus callosum; HP:0003593, Infantile onset; HP:0000486, Strabismus; HP:0002305, Athetosis; HP:0000253, Progressive microcephaly; HP:0007359, Focal-onset seizure; HP:0001284, Areflexia; HP:0011344, Severe global de

In [ ]:
#instalar y probar HPOSim
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("AnnotationDbi")
require(devtools)
install_github('cran/HPO.db')
library(HPO.db)
require(devtools)
install_github('cran/HPOSim')
library(HPOSim)
library(HPOSim)
#similitud semantica de Resnik entre dos términos
.initialize()
IC<-get("termIC", envir=HPOSimEnv)
calcTermSim("HP:0000028","HP:0000033", method = "Wang", IC, verbose = FALSE)
# similitud semnántica de Resnik entre un set de términos HPO
.initialize()
IC<-get("termIC",envir=HPOSimEnv)
getTermListSim(c("HP:0000006", "HP:00109709"),
c("HP:0004469","HP:0011949","HP:0030877","HP:0000083","HP:0011740","HP:0001949","HP:0011947","HP:0002097","HP:0002094","HP:0200117","HP:0001824","HP:0100812","HP:0003351","HP:0001942","HP:0001217","HP:0100749","HP:0031274","HP:0002900","HP:0000006","HP:0001047","HP:0005376","HP:0002153","HP:0000822","HP:0012092","HP:0040085","HP:0000848","HP:0012236","HP:0012735","HP:0002013","HP:0002637","HP:0008872","HP:0012378","HP:0004319","HP:0011675","HP:0002783","HP:0001531","HP:0001658","HP:0002110","HP:0200114","HP:0002902","HP:0001945","HP:0002105","HP:0003508","HP:0001324","HP:0030830","HP:0011110","HP:0000112","HP:0001081","HP:0004326","HP:0030828","HP:0002795","HP:0200039","HP:0001944","HP:0002019","HP:0002754","HP:0031245"),
combinemethod="mean", method="Resnik", IC=IC, verbose=FALSE)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.17 (BiocManager 1.30.20), R 4.3.0 (2023-04-21)

Installing package(s) 'BiocVersion', 'AnnotationDbi'

also installing the dependencies ‘bitops’, ‘zlibbioc’, ‘RCurl’, ‘GenomeInfoDbData’, ‘XVector’, ‘GenomeInfoDb’, ‘plogr’, ‘png’, ‘Biostrings’, ‘BiocGenerics’, ‘Biobase’, ‘IRanges’, ‘RSQLite’, ‘S4Vectors’, ‘KEGGREST’


Old packages: 'bit', 'bslib', 'curl', 'devtools', 'digest', 'gert', 'highr',
  'isoband', 'jsonlite', 'pkgload', 'roxygen2', 'tidyverse', 'usethis',
  'whisker', 'boot', 'foreign'

Loading required package: devtools

Loading required package: usethis




Skipping 1 packages not available: AnnotationDbi



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpdekkPo/remotes19d7f8c562e/cran-HPO.db-48d4c3b/DESCRIPTION’ ... OK
* preparing ‘HPO.db’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘HPO.db_1.2.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'brows

igraph (NA -> 1.4.3) [CRAN]


Skipping 1 packages not available: AnnotationDbi

Installing 1 packages: igraph

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpdekkPo/remotes19d58e66d6e/cran-HPOSim-3f65384/DESCRIPTION’ ... OK
* preparing ‘HPOSim’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* looking to see if a ‘data/datalist’ file should be added
* building ‘HPOSim_1.2.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

initializing HPOSim package ...

done.

initializing HPOSim package ...

done.



[1] 0.5463333

[1] 0.007930995

In [ ]:
# Cargar la biblioteca de HPOsim
library(HPOSim)

# Inicializar HPOsim
.initialize()

# Obtener el término IC de HPOsim
IC <- get("termIC", envir = HPOSimEnv)

# Definir la lista de códigos HPO de entrada
input_hpos <- c("HP:0002099")

# Definir una función para calcular la similitud semántica para cada fila del dataframe
get_hposim <- function(row) {
  # Obtener los códigos HPO de la columna "HPO codes gene" de la fila actual
  hpos <- unlist(strsplit(as.character(row["HPO codes gene"]), ", "))
  # Calcular la similitud semántica de entre los códigos HPO de entrada y los de la fila actual
  sim <- getTermListSim(hpos, input_hpos, combinemethod = "funSimAvg", method = "Lin", IC = IC, verbose = FALSE)
  return(sim)
}

# Aplicar la función a cada fila del dataframe y almacenar los resultados en una nueva columna
merged_df$HPOSIM <- apply(merged_df, 1, get_hposim)
merged_df$HPOSIM

Warning message in getTermListSim(hpos, input_hpos, combinemethod = "funSimAvg", :
“No HPO information for  HP:0002099 . Similarity set to NaN.”
Warning message in getTermListSim(hpos, input_hpos, combinemethod = "funSimAvg", :
“No HPO information for  HP:0002099 . Similarity set to NaN.”
Warning message in getTermListSim(hpos, input_hpos, combinemethod = "funSimAvg", :
“No HPO information for  HP:0002099 . Similarity set to NaN.”
Warning message in getTermListSim(hpos, input_hpos, combinemethod = "funSimAvg", :
“No HPO information for  HP:0002099 . Similarity set to NaN.”
Warning message in getTermListSim(hpos, input_hpos, combinemethod = "funSimAvg", :
“No HPO information for  HP:0002099 . Similarity set to NaN.”
Warning message in getTermListSim(hpos, input_hpos, combinemethod = "funSimAvg", :
“No HPO information for  HP:0002099 . Similarity set to NaN.”
Warning message in getTermListSim(hpos, input_hpos, combinemethod = "funSimAvg", :
“No HPO information for  HP:0002099 . Similarit

[1] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000       NaN 0.2404090
  [8] 0.6000000       NaN 0.2725166 0.2725166 0.0000000 0.0000000 0.0000000
 [15]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
 [22]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
 [29]       NaN       NaN 0.0000000 0.2347570 0.2347570 0.2347570 0.2347570
 [36] 0.2347570 0.0000000       NaN       NaN       NaN       NaN       NaN
 [43]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
 [50]       NaN       NaN 0.0000000 0.5186155 0.5186155 0.5186155 0.5186155
 [57]       NaN       NaN 0.2811279 0.2811279 0.2811279 0.2811279       NaN
 [64]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
 [71]       NaN       NaN       NaN       NaN 0.5697901       NaN       NaN
 [78]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
 [85]       NaN       NaN       NaN 0.0000000 0.0000000 0.0000000 0.0000000
 [92] 0.0000000 0.0000000       NaN 0.2243411 0.0000000 0.5173167 0.5173167
 [99] 0.5173167       NaN       NaN       NaN       NaN       NaN       NaN
[106] 0.0000000 0.0000000 0.0000000 0.2802111 0.2802111 0.2802111 0.1641606
[113] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000       NaN
[120]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
[127] 0.1187943       NaN       NaN 0.2948071 0.2948071       NaN       NaN
[134]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
[141]       NaN       NaN       NaN 0.5085610 0.5085610       NaN 0.0000000
[148] 0.0000000       NaN       NaN       NaN       NaN       NaN       NaN
[155]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
[162]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
[169]       NaN 0.5384615 0.1971823 0.1971823 0.1971823 0.1971823 0.1971823
[176]       NaN       NaN       NaN 0.0000000       NaN 0.1218966       NaN
[183]       NaN       NaN       NaN 0.0000000 0.0000000       NaN       NaN
[190] 0.1799134 0.1799134       NaN       NaN       NaN       NaN       NaN
[197]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
[204]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
[211]       NaN 0.0000000 0.0000000 0.0000000       NaN 0.1079346 0.1079346
[218] 0.1079346 0.0000000 0.0000000 0.3215083 0.3215083 0.1905042 0.1905042
[225] 0.1905042       NaN       NaN       NaN       NaN       NaN       NaN
[232]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
[239]       NaN       NaN 0.2332520       NaN 0.2327234 0.2327234       NaN
[246]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
[253]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
[260]       NaN       NaN       NaN       NaN       NaN       NaN 0.6176651
[267]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
[274]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
[281]       NaN       NaN       NaN       NaN       NaN       NaN 0.0000000
[288] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000       NaN 0.6685326
[295]       NaN       NaN       NaN 0.3473991 0.3473991       NaN       NaN
[302]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
[309]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
[316]       NaN       NaN       NaN 0.0000000 0.0000000       NaN       NaN
[323]       NaN       NaN       NaN       NaN       NaN       NaN       NaN
[330] 0.0000000 0.1479277 0.1479277 0.1479277 0.1479277 0.1479277 0.1730950
[337] 0.1730950       NaN       NaN       NaN       NaN       NaN       NaN
[344]       NaN       NaN       NaN       NaN       NaN 0.0000000       NaN
[351] 0.0000000       NaN       NaN       NaN       NaN       NaN       NaN
[358]       NaN       NaN       NaN       NaN       NaN       NaN 0.0000000
[365]       NaN 0.0000000       NaN       NaN     

In [ ]:
#Definir los HPO de inputs
input_hpos <- c("HP:0002099")
# Definir una función para buscar una coincidencia entre los códigos HPO y el patrón de búsqueda
find_hpo_match <- function(row, input_hpos) {
  # Obtener los códigos HPO de la columna "HPO codes gene" de la fila actual
  hpos <- unlist(strsplit(as.character(row["HPO codes gene"]), ", "))
  # Buscar los códigos HPO que son comunes en la columna "HPO codes gene" y en la lista de códigos HPO de entrada
  common_hpos <- intersect(hpos, input_hpos)
  # Devolver una cadena de texto con los códigos HPO comunes separados por comas
  if (length(common_hpos) > 0) {
    return(paste(common_hpos, collapse = ","))
  } else {
    return("")
  }
}

# Aplicar la función a cada fila del dataframe y almacenar los resultados en una nueva columna
merged_df$HPOmatch <- apply(merged_df, 1, find_hpo_match, input_hpos = input_hpos)
head(merged_df)





,Gene.refGene,Zygosity,Transcript_exon_HGVSc._HGVSp.,dbSNP,Chr,Start,End,Ref,Alt,Func.refGene,⋯,gnomAD_genome_EAS,gnomAD_genome_FIN,gnomAD_genome_NFE,gnomAD_genome_OTH,hpo,omim_orpha,HPO codes gene,HPOSIM,HPOmatch,HPOmatch_desc
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
1,A4GALT,heterozygous,"A4GALT:NM_001318038:exon3:c.A109G:p.M37V,A4GALT:NM_017436:exon3:c.A109G:p.M37V",rs11541159,chr22,43089849,43089849,T,C,exonic,⋯,0.1434,0.2883,0.3477,0.3286,"HP:0000006, Autosomal dominant inheritance; HP:0010970, Blood group antigen abnormality",OMIM:111400; OMIM:111400,"HP:0000006, HP:0010970",0,,
2,A4GALT,heterozygous,"A4GALT:NM_001318038:exon3:c.G987A:p.T329T,A4GALT:NM_017436:exon3:c.G987A:p.T329T",rs9623659,chr22,43088971,43088971,C,T,exonic,⋯,0.1422,0.2886,0.3480,0.3272,"HP:0000006, Autosomal dominant inheritance; HP:0010970, Blood group antigen abnormality",OMIM:111400; OMIM:111400,"HP:0000006, HP:0010970",0,,
3,A4GALT,heterozygous,"A4GALT:NM_001318038:exon3:c.C903G:p.P301P,A4GALT:NM_017436:exon3:c.C903G:p.P301P",rs6002904,chr22,43089055,43089055,G,C,exonic,⋯,0.7171,0.5956,0.5703,0.5932,"HP:0000006, Autosomal dominant inheritance; HP:0010970, Blood group antigen abnormality",OMIM:111400; OMIM:111400,"HP:0000006, HP:0010970",0,,
4,ACO2,heterozygous,ACO2:NM_001098:exon5:c.C670T:p.L224L,rs1799932,chr22,41911525,41911525,C,T,exonic,⋯,0.1634,0.4417,0.5030,0.4436,"HP:0000007, Autosomal recessive inheritance; HP:0003621, Juvenile onset; HP:0000543, Optic disc pallor; HP:0007663, Reduced visual acuity; HP:0030528, Paracentral scotoma; HP:0000648, Optic atrophy; HP:0000642, Red-green dyschromatopsia; HP:0011463, Childhood onset; HP:0001252, Hypotonia; HP:0002079, Hypoplasia of the corpus callosum; HP:0003593, Infantile onset; HP:0000486, Strabismus; HP:0002305, Athetosis; HP:0000253, Progressive microcephaly; HP:0007359, Focal-onset seizure; HP:0001284, Areflexia; HP:0011344, Severe global developmental delay; HP:0000556, Retinal dystrophy; HP:0000007, Autosomal recessive inheritance; HP:0000407, Sensorineural hearing impairment; HP:0000648, Optic atrophy; HP:0001251, Ataxia; HP:0010864, Intellectual disability, severe; HP:0001272, Cerebellar atrophy; HP:0007108, Demyelinating peripheral neuropathy; HP:0002120, Cerebral cortical atrophy; HP:0001265, Hyporeflexia; HP:0002069, Bilateral tonic-clonic seizure; HP:0001508, Failure to thrive; HP:0000639, Nystagmus",OMIM:616289; OMIM:616289; OMIM:616289; OMIM:616289; OMIM:616289; OMIM:616289; OMIM:616289; OMIM:616289; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559; OMIM:614559,"HP:0000007, HP:0003621, HP:0000543, HP:0007663, HP:0030528, HP:0000648, HP:0000642, HP:0011463, HP:0001252, HP:0002079, HP:0003593, HP:0000486, HP:0002305, HP:0000253, HP:0007359, HP:0001284, HP:0011344, HP:0000556, HP:0000007, HP:0000407, HP:0000648, HP:0001251, HP:0010864, HP:0001272, HP:0007108, HP:0002120, HP:0001265, HP:0002069, HP:0001508, HP:0000639",0,,
5,ACO2,heterozygous,ACO2:NM_001098:exon3:c.A192C:p.T64T,rs137831,chr22,41903813,41903813,A,C,exonic,⋯,0.5795,0.2552,0.1987,0.2520,"HP:0000007, Autosomal recessive inheritance; HP:0003621, Juvenile onset; HP:0000543, Optic disc pallor; HP:0007663, Reduced visual acuity; HP:0030528, Paracentral scotoma; HP:0000648, Optic atrophy; HP:0000642, Red-green dyschromatopsia; HP:0011463, Childhood onset; HP:0001252, Hypotonia; HP:0002079, Hypoplasia of the corpus callosum; HP:0003593, Infantile onset; HP:0000486, Strabismus; HP:0002305, Athetosis; HP:0000253, Progressive microcephaly; HP:0007359, Focal-onset seizure; HP:0001284, Areflexia; HP:0011344, Severe global developmental delay; HP:0000556, Retinal dystrophy; HP:0000007, Autosomal recessive inheritance; HP:0000407, Sensor

In [ ]:
# Leer el archivo genes_to_phenotype.txt
genes_pheno2 <- read.delim("/content/genes_to_phenotype.txt", header=FALSE, comment.char="#")

# Seleccionar solo las columnas necesarias
genes_pheno_sub2 <- genes_pheno2[, c("V2", "V3", "V4", "V9")]

# Renombrar las columnas
colnames(genes_pheno_sub2) <- c("gene_id", "hpo_id", "hpo_desc", "omim_orpha")

# Dividir la columna HPOmatch en filas
hpo_rows <- strsplit(as.character(merged_df$HPOmatch), ",")

# Crear un vector para almacenar las descripciones
desc_vec <- character(length(hpo_rows))

# Buscar la descripción de cada HPO code
for (i in seq_along(hpo_rows)) {
  # Filtrar la tabla para obtener solo las filas correspondientes a los HPOs en x
  filtered_table <- genes_pheno_sub2[genes_pheno_sub2$hpo_id %in% hpo_rows[[i]],]
  # Unir todas las descripciones en una sola cadena
  desc_string <- paste(unique(filtered_table$hpo_desc), collapse = ", ")
  # Almacenar la cadena en el vector de descripciones
  desc_vec[i] <- desc_string
}

# Agregar el vector de descripciones a la columna HPOmatch_desc del dataframe
merged_df$HPOmatch_desc <- desc_vec


In [ ]:
#filtrar por los HPO de interés
input_hpo_codes <- c("HP:0002099")
filtered_df <- merged_df[apply(merged_df[, "HPO codes gene", drop=FALSE], 1, function(x) any(grepl(paste(input_hpo_codes, collapse="|"), x))), ]
#se transforma el archivo filtered_df en un archivo cvs
write.csv(filtered_df, "filtered_df.csv", row.names = FALSE)


In [ ]:
# Seleccionar las columnas de interés
columnas_interes <- c("Gene.refGene", "Zygosity", "Transcript_exon_HGVSc._HGVSp.", "ExonicFunc.refGene", "HPOSIM", "HPOmatch", "HPOmatch_desc")
filtered_df_seleccionado <- filtered_df[columnas_interes]

# Filtrar las filas que no sean synonymous SNV en la columna ExonicFunc.refGene
filtered_df_filtrado <- filtered_df_seleccionado[filtered_df_seleccionado$ExonicFunc.refGene != "synonymous SNV", ]

# Ordenar la columna HPOSIM_Resnik de mayor a menor
filtered_df_filtrado <- filtered_df_filtrado[order(filtered_df_filtrado$HPOSIM, decreasing = TRUE), ]

# Mostrar el DataFrame filtrado y ordenado
filtered_df_filtrado


,Gene.refGene,Zygosity,Transcript_exon_HGVSc._HGVSp.,ExonicFunc.refGene,HPOSIM,HPOmatch,HPOmatch_desc
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
294,PTGER2,homozygous,PTGER2:NM_000956:exon1:c.G5A:p.G2D,nonsynonymous SNV,0.6685326,HP:0002099,Asthma
391,TBX21,homozygous,TBX21:NM_013351:exon1:c.G83A:p.G28D,nonsynonymous SNV,0.6478573,HP:0002099,Asthma
266,PLA2G7,homozygous,"PLA2G7:NM_001168357:exon9:c.A797G:p.K266R,PLA2G7:NM_005084:exon9:c.A797G:p.K266R",nonsynonymous SNV,0.6176651,HP:0002099,Asthma
8,ADRB2,homozygous,ADRB2:NM_000024:exon1:c.G420C:p.K140N,nonsynonymous SNV,0.6000000,HP:0002099,Asthma
75,CARD10,heterozygous,CARD10:NM_014550:exon4:c.789_794del:p.K272_E273del,nonframeshift deletion,0.5697901,HP:0002099,Asthma
170,HNMT,homozygous,"HNMT:NM_001024074:exon1:c.G125A:p.G42D,HNMT:NM_001024075:exon1:c.G125A:p.G42D,HNMT:NM_006895:exon1:c.G125A:p.G42D",nonsynonymous SNV,0.5384615,HP:0002099,Asthma
53,ARVCF,heterozygous,ARVCF:NM_001670:exon5:c.T524C:p.V175A,nonsynonymous SNV,0.5186155,HP:0002099,Asthma
54,ARVCF,homozygous,ARVCF:NM_001670:exon18:c.G2717A:p.R906Q,nonsynonymous SNV,0.5186155,HP:0002099,Asthma
55,ARVCF,homozygous,ARVCF:NM_001670:exon5:c.C659T:p.P220L,nonsynonymous SNV,0.5186155,HP:0002099,Asthma


In [ ]:
# Seleccionar las columnas de interés
columnas_interes <- c("Gene.refGene", "Zygosity", "ExonicFunc.refGene", "HPOSIM", "HPOmatch", "HPOmatch_desc")
filtered_df_seleccionado <- filtered_df[columnas_interes]

# Filtrar las filas que no sean synonymous SNV en la columna ExonicFunc.refGene
filtered_df_filtrado <- filtered_df_seleccionado[filtered_df_seleccionado$ExonicFunc.refGene != "synonymous SNV", ]

# Ordenar la columna HPOSIM_Resnik de mayor a menor
filtered_df_filtrado <- filtered_df_filtrado[order(filtered_df_filtrado$HPOSIM, decreasing = TRUE), ]

# Mostrar el DataFrame filtrado y ordenado
filtered_df_filtrado

,Gene.refGene,Zygosity,ExonicFunc.refGene,HPOSIM,HPOmatch,HPOmatch_desc
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
294,PTGER2,homozygous,nonsynonymous SNV,0.6685326,HP:0002099,Asthma
391,TBX21,homozygous,nonsynonymous SNV,0.6478573,HP:0002099,Asthma
266,PLA2G7,homozygous,nonsynonymous SNV,0.6176651,HP:0002099,Asthma
8,ADRB2,homozygous,nonsynonymous SNV,0.6000000,HP:0002099,Asthma
75,CARD10,heterozygous,nonframeshift deletion,0.5697901,HP:0002099,Asthma
170,HNMT,homozygous,nonsynonymous SNV,0.5384615,HP:0002099,Asthma
53,ARVCF,heterozygous,nonsynonymous SNV,0.5186155,HP:0002099,Asthma
54,ARVCF,homozygous,nonsynonymous SNV,0.5186155,HP:0002099,Asthma
55,ARVCF,homozygous,nonsynonymous SNV,0.5186155,HP:0002099,Asthma
